# Importacion librerias

In [9]:
!pip install gurobipy
from gurobipy import *
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Definicion de parametros

In [10]:
# Cargar los datos desde el archivo Excel
df_costos = pd.read_excel('info.xlsx', sheet_name="Hoja 1", usecols="A:C", skiprows=0, nrows=96)
df_costos['Hora'] = pd.to_datetime(df_costos['Hora'])


# Definición de Nodos
nodos = ["Parqueadero", "Panzenu", "Pradera"]
clientes = nodos[1:]
n = len(nodos)

# Crear los arcos entre todos los nodos menos los que son del mismo nodo
arcos = [(i, j) for i in nodos for j in nodos if i != j]

# Tiempos de servicio manuales en cada nodo
s = {"Parqueadero": 0, "Panzenu": 0, "Pradera": 0}

# Definición de tiempos de viaje manuales entre nodos
tiempo = {
    ("Parqueadero", "Panzenu"): 50,
    ("Parqueadero", "Pradera"): 20,
    ("Panzenu", "Parqueadero"): 50,
    ("Panzenu", "Pradera"): 30,
    ("Pradera", "Parqueadero"): 20,
    ("Pradera", "Panzenu"): 30,
}

# Definición de Vehículos con capacidades
num_buses = 6
vehiculos = list(range(1, num_buses + 1))
Q = {k: 50 for k in vehiculos}

# Funciones complementarias

In [11]:
#Funcion para obtener la demanda del nodo a donde se dirige el bus
def get_demand(timestamp, df):
    timestamp_str = timestamp.strftime('2024-08-05 %H:%M:%S')
    demanda_row = df[df['Hora'] == timestamp_str]
    if not demanda_row.empty:
        panzenu_demand = demanda_row['Panzenu '].values[0]
        pradera_demand = demanda_row['Pradera '].values[0]
        return panzenu_demand, pradera_demand
    else:
        return 0, 0

# Modelo

In [12]:

# Variables de decisión
arco_var = [(i, j, k) for i in nodos for j in nodos for k in vehiculos if i != j]
arco_tiempos = [(i, k) for i in nodos for k in vehiculos]

# Crear el modelo
model = Model('VRPTW')

# Variables de Decisión
x = model.addVars(arco_var, vtype=GRB.BINARY, name='x')
t = model.addVars(arco_tiempos, vtype=GRB.CONTINUOUS, name='t')

# Función Objetivo: Minimizar el tiempo total de viaje
model.setObjective(quicksum(tiempo[i, j] * x[i, j, k] for i, j, k in arco_var), GRB.MINIMIZE)

# Restricciones
# Salida y llegada al Parqueadero
model.addConstrs(quicksum(x["Parqueadero", j, k] for j in clientes) <= 1 for k in vehiculos)
model.addConstrs(quicksum(x[i, "Parqueadero", k] for i in clientes) <= 1 for k in vehiculos)

# Cada cliente debe ser visitado una o mas beses de acuerdo al horario
model.addConstrs(quicksum(x[i, j, k] for j in nodos for k in vehiculos if i != j) >= 1 for i in clientes)

# Conservación de flujo
model.addConstrs(
    quicksum(x[i, j, k] for j in nodos if i != j) - quicksum(x[j, i, k] for j in nodos if i != j) == 0
    for i in nodos for k in vehiculos
)

# Restricción de capacidad de los vehículos de acuerdo a la demanda de esa hora
# Restricción de capacidad de los vehículos
model.addConstrs(
    (
        quicksum(
            get_demand(tiempo_llegada_timestamp, df_costos)[0] * x[i, j, k]
            for i in nodos if i != "Parqueadero" for j in nodos if j == "Panzenu" and i != j
        ) +
        quicksum(
            get_demand(tiempo_llegada_timestamp, df_costos)[1] * x[i, j, k]
            for i in nodos if i != "Parqueadero" for j in nodos if j == "Pradera" and i != j
        ) <= Q[k]
    )
    for k in vehiculos
)



# Restricción de ventana de tiempo(asegura que el la hora a la cual llega el bus coincida con el tiempo transcurrido)
model.addConstrs(
    (x[i, j, k] == 1) >> (t[i, k] + s[i] + tiempo[i, j] == t[j, k])
    for i in clientes for j in clientes for k in vehiculos if i != j
)


{('Panzenu', 'Pradera', 1): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Panzenu', 'Pradera', 2): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Panzenu', 'Pradera', 3): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Panzenu', 'Pradera', 4): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Panzenu', 'Pradera', 5): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Panzenu', 'Pradera', 6): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Pradera', 'Panzenu', 1): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Pradera', 'Panzenu', 2): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Pradera', 'Panzenu', 3): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Pradera', 'Panzenu', 4): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Pradera', 'Panzenu', 5): <gurobi.GenConstr *Awaiting Model Update*>,
 ('Pradera', 'Panzenu', 6): <gurobi.GenConstr *Awaiting Model Update*>}

In [13]:
# Optimizar el modelo
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 38 rows, 54 columns and 132 nonzeros
Model fingerprint: 0x0e83e2a6
Model has 12 general constraints
Variable types: 18 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [2e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
  GenCon rhs range [3e+01, 3e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve added 6 rows and 0 columns
Presolve removed 0 rows and 6 columns
Presolve time: 0.00s
Presolved: 44 rows, 48 columns, 144 nonzeros
Presolved model has 12 SOS constraint(s)
Variable types: 12 continuous, 36 integer (36 binary)
Found heuristic solution: objective 100.0000000

Root relaxation: objective 6.000000e+01, 14 iterations, 0.00 seconds (0.00 wo

# Resultados modelo

In [14]:
# Hora límite de regreso al Parqueadero (8:50 PM)
hora_limite = 20 * 60 + 50  # 20:50 en minutos

# Definir la ruta para cada vehículo
rutas = {
    1: ["Parqueadero", "Pradera", "Panzenu"],
    2: ["Parqueadero", "Panzenu", "Pradera"]
}

# Crear una lista para almacenar los resultados
resultados = []

# Mostrar los tiempos de salida y llegada para cada tramo de la ruta de cada vehículo
for vehiculo_id in [1, 2]:
    r = rutas[vehiculo_id]
    print(f"\nRuta del vehículo {vehiculo_id}:")

    # Hora de salida inicial para el primer vehículo
    tiempo_actual = 5 * 60

    # Comenzar en el Parqueadero
    nodo_actual = r[0]

    #Definir a cual paradero seguir dependiendo donde se encuentre
    while True:
        if nodo_actual == "Parqueadero":
            nodo_siguiente = r[1]
        elif nodo_actual == "Pradera":
            nodo_siguiente = "Panzenu"
        elif nodo_actual == "Panzenu":
            nodo_siguiente = "Pradera"
        else:
            break

        #Actualizacion de la hora teniendo en cuenta el tiempo de transporte
        hora_salida_horas = int(tiempo_actual // 60)
        minutos_salida = int(tiempo_actual % 60)
        tiempo_actual += tiempo[nodo_actual, nodo_siguiente]
        tiempo_actual %= 1440
        hora_llegada_horas = int(tiempo_actual // 60)
        minutos_llegada = int(tiempo_actual % 60)
        tiempo_llegada_timestamp = pd.Timestamp("2024-08-05") + pd.Timedelta(hours=hora_llegada_horas, minutes=minutos_llegada)
        #Obtencion demanda dependiendo de la hora a la que llega el bus al destino
        try:
            panzenu_demand, pradera_demand = get_demand(tiempo_llegada_timestamp, df_costos)
            if nodo_siguiente == "Panzenu":
                demanda = f"Panzenu: {panzenu_demand}"
                demanda1=panzenu_demand
            elif nodo_siguiente == "Pradera":
                demanda = f"Pradera: {pradera_demand}"
                demanda1=pradera_demand
            else:
                demanda = "Sin demanda"

        except KeyError as e:
            print(f"Error al obtener demanda para el tiempo {tiempo_llegada_timestamp}: {e}")
            demanda = "Demanda no disponible"

        # Imprimir la ruta y la demanda
        print(f"Desde {nodo_actual} a {nodo_siguiente}: Salida {hora_salida_horas:02d}:{minutos_salida:02d}, "
              f"Llegada {hora_llegada_horas:02d}:{minutos_llegada:02d}, {demanda}")

        # Almacenar los resultados en la lista
        resultados.append({
            "Vehículo": vehiculo_id,
            "Desde": nodo_actual,
            "Hasta": nodo_siguiente,
            "Hora Salida": f"{hora_salida_horas:02d}:{minutos_salida:02d}",
            "Hora Llegada": f"{hora_llegada_horas:02d}:{minutos_llegada:02d}",
            "Demanda": demanda1
        })

        #Almacenamiento informacion cuando llega al parqueadero y termina ruta
        if tiempo_actual >= hora_limite:
            print(f"Regresando al Parqueadero desde {nodo_siguiente}.")

            tiempo_actual += tiempo[nodo_siguiente, "Parqueadero"]
            tiempo_actual %= 1440
            hora_regreso_horas = int(tiempo_actual // 60)
            minutos_regreso = int(tiempo_actual % 60)
            resultados.append({
                "Vehículo": vehiculo_id,
                "Desde": nodo_siguiente,
                "Hasta": "Parqueadero",
                "Hora Salida": f"{hora_llegada_horas:02d}:{minutos_llegada:02d}",
                "Hora Llegada": f"{hora_regreso_horas:02d}:{minutos_regreso:02d}",
                "Demanda": "Regreso"
            })
            print(f"Desde {nodo_siguiente} a Parqueadero: Salida {hora_llegada_horas:02d}:{minutos_llegada:02d}, "
                  f"Llegada {hora_regreso_horas:02d}:{minutos_regreso:02d}, Regreso")
            break
        # Actualizar nodo_actual
        nodo_actual = nodo_siguiente


Ruta del vehículo 1:
Desde Parqueadero a Pradera: Salida 05:00, Llegada 05:20, Pradera: 36
Desde Pradera a Panzenu: Salida 05:20, Llegada 05:50, Panzenu: 27
Desde Panzenu a Pradera: Salida 05:50, Llegada 06:20, Pradera: 39
Desde Pradera a Panzenu: Salida 06:20, Llegada 06:50, Panzenu: 36
Desde Panzenu a Pradera: Salida 06:50, Llegada 07:20, Pradera: 41
Desde Pradera a Panzenu: Salida 07:20, Llegada 07:50, Panzenu: 41
Desde Panzenu a Pradera: Salida 07:50, Llegada 08:20, Pradera: 42
Desde Pradera a Panzenu: Salida 08:20, Llegada 08:50, Panzenu: 43
Desde Panzenu a Pradera: Salida 08:50, Llegada 09:20, Pradera: 42
Desde Pradera a Panzenu: Salida 09:20, Llegada 09:50, Panzenu: 42
Desde Panzenu a Pradera: Salida 09:50, Llegada 10:20, Pradera: 41
Desde Pradera a Panzenu: Salida 10:20, Llegada 10:50, Panzenu: 41
Desde Panzenu a Pradera: Salida 10:50, Llegada 11:20, Pradera: 41
Desde Pradera a Panzenu: Salida 11:20, Llegada 11:50, Panzenu: 40
Desde Panzenu a Pradera: Salida 11:50, Llegada 12:

In [15]:
# Crear un DataFrame de los resultados
df_resultados = pd.DataFrame(resultados)
print("\nResultados de las Rutas:")
print(df_resultados)


Resultados de las Rutas:
    Vehículo        Desde        Hasta Hora Salida Hora Llegada  Demanda
0          1  Parqueadero      Pradera       05:00        05:20       36
1          1      Pradera      Panzenu       05:20        05:50       27
2          1      Panzenu      Pradera       05:50        06:20       39
3          1      Pradera      Panzenu       06:20        06:50       36
4          1      Panzenu      Pradera       06:50        07:20       41
..       ...          ...          ...         ...          ...      ...
60         2      Panzenu      Pradera       18:50        19:20       36
61         2      Pradera      Panzenu       19:20        19:50       26
62         2      Panzenu      Pradera       19:50        20:20       34
63         2      Pradera      Panzenu       20:20        20:50       19
64         2      Panzenu  Parqueadero       20:50        21:40  Regreso

[65 rows x 6 columns]


# Exportacion datos

In [16]:
#Ejecutar para crear excel de la tabla de resultados de transporte
nombre_archivo = 'resultados_rutas.xlsx'
df_resultados.to_excel(nombre_archivo, index=False)

print(f"Los resultados se han exportado a {nombre_archivo}")

Los resultados se han exportado a resultados_rutas.xlsx
